# MFCC - Mel frequency ceptral coefficients

![](image12.png)

Ta có thể hình dung việc tính MFCC theo luồng xử lý: cắt chuỗi tín hiệu âm thanh thành các đoạn ngắn bằng nhau (25ms) và overlap lên nhau (10ms). Mỗi đoạn âm thanh này được biến đổi, tính toán để thu được 39 features. Lỗi list 39 features có tính độc lập cao, ít nhiễu, đủ nhỏ để đảm bảo tính toán, đủ thông tin để đảm bảo chất lượng cho các thuật toán nhận dạng

![](image1.png)

Đây là hình mô tả luồng xử lý từ âm thanh đầu vào $\rightarrow$ MFCC

# 1. A/D Conversion and Pre-emphasis

## 1.1 AD Conversion

Âm thanh là dạng tín hiệu liên tục, trong khi iddos máy tính làm việc với các con số rời rạc. Ta cần lấy mẫu tại các khoảng thời gian cách đều nhau với 1 tần số lấy mẫu xác định (sr) để chuyển từ dạn gtins hiệu liên tục về dạng rời rạc. Vd: sample_rate(8000) $\rightarrow$ 1s lấy 8000 giá trị.

![](image2.png)

Tai người nghe được âm thanh trong khoảng 20Hz $\rightarrow$ 20.000 Hz. Theo định lý lấy mẫu Nyquist-Shannon: với 1 tín hiệu có các tần số thành phần $\leq$ $f_{m}$, để đảm bảo việc lấy mâix không làm mất mát thông tin (aliasing), tần số lấy mẫu $f_{s}$ phỉa đảm bảo $f_{s} \geq 2f_{m}$.

Vậy để đảm bảo việc lấy mẫu không làm mất mát thông tin, tần số lấy mẫu $f_{s}$ = 44100 Hz. Tuy nhiên trong nhiều trường hợp, người ta chỉ cần lấy $f_{s}$ = 8000 Hz hoặc $f_{s}$ = 16000 Hz.

## 1.2 Pre-emphasis

Do đặc điểm cấu tạo thanh quản và các bộ phận phát âm nên tiếng nói của hcungs ta có đặc điểm: các âm ở tần số thấp có mức năng lượng cao, các âm ở tần số cao lại có mức năng lượng khá thấp. Trong khi đó, các tần số cao này vẫn chứa nhiều thông tin về âm vị. Vì vậy chúng ta cần 1 bước pre-emphasis để kích các tín hiệu ở tần số cao này lên.

![](image3.png)

# 2. Spectrogram

## 2.1 Windowing

Thay vì biến đổi Fourier trên cả đoạn âm thanh dài, ta trượt 1 cửa sổ dọc theo tín hiệu để lấy ra các frame rồi mới áp dung DFT trên từng frame này (DFT - Discrete Fourier Transform). Tốc độ nói của con người trung bình khoảng 3,4 từ mỗi giây, mỗi từ khoảng 3-4 âm, mỗi âm chia thành 3-4 phần, như vậy 1 giây âm thanh được chia thành 36-40 phần, ta chọn độ rộng mỗi frame khoảng 20-25ms là vừa đủ rộng để bao 1 phần âm thanh. Các frame được overlap lên nhau khoảng 10ms để có thể capture lại sự thay đổi context.

![](image4.png)

Tuy nhiên, việc cắt frame sẽ làm các giá trị ở 2 biên của frame bị giảm đột ngồ (về giá trị 0), sẽ dẫn tới hiện tượng: khi DFT sang miền tần số sẽ có rất nhiều nhiễu ở tần số cao. Để khắc phục điều này, ta cần làm mượt bằng cách nhân chập frame với 1 vài loại window. Có 1 vài loại window phổ biến là Hamming window, Hanning window ... có tác dụng làm giá trị biên frame giảm xuống từ từ.

![](image5.png)

Hình dưới đây sẽ cho ta thấy rõ được tác dụng của các window này. Trong các hình nhỏ:
- hình 1 là frame được cắt ra từ âm thanh gốc, âm thanh gốc là sự kết hợp của 2 sóng.
- hình 2 nếu áp dụng rectangle window (tức là cắt trực tiếp), tín hiệu miền tần số tương ứng là hình 3. Ta có thể thấy tín hiệu chứa rất nhiều nhiễu. Nếu áp dụng các window như Hanning, Hamming, Blackman, tín hiệu miền tần số thu được khá mượt.

![](image6.png)

## 2.2 DFT

Trên từng frame, ta áp dụng DFT - Discrete Fourier Transform theo công thức:

$$
X[k] = \sum^{N - 1}_{n = 0}{x[n].exp(-j\frac{2\pi}{N}kn)}
$$

Mỗi frame ta thu được 1 list các giá trị độ lớn (magnitude) tương ứng với từng tần số từ 0 $\rightarrow$ N. Áp dụng trên tất cả các frame, ta đã thu được 1 Spectrogram như hình dứoi. Trục x là thời gian (tương ứng với thứ tự các frame), trục y thể hiện dải tần số từ 0 $\rightarrow$ 10000 Hz, giá trị magnitude tại từng tần số được thể hiện bằng màu sắc. Qua quan sát spectrogram này, ta nhận thấy tại các tần số thấp thường có magnitude cao, tần số cao thường có magnitude thấp.

![](image7.png)

Hình dưới là các spectrogram của 4 nguyên âm. Quan sát spectrogram lần lượt từ dưới lên, người ta nhận thấy có 1 vài tần số đặc trưng gọi là các formant, gọi là các tần số F1, F2, F3, ... Các chuyên gia về ngữ âm học có thể dựa vào vị trí, thời gian, sự thay đổi các formant trên spectrogram để xác định đoạn âm thanh đó là của âm vị nào.

![](image8.png)

Như vậy ta đã biêt cách tạo ra spectrogram. Tuy nhiên trong nhiều bài toán (đặc biệt là speech recognition), spectrogram không phải là sự lựa chọn hoàn hảo. Vì vậy ta cần thêm vài bước tính nữa để thu được MFCC, tốt hơn, phổ biến hơn, hiệu quả hơn spectrogram.

# 3. Mel filterbank

Như mình đã mô tả ở phần trước, cách cảm nhận của tai người là phi tuyến tính, không giống các thiết bị đo. Tai người cảm nhận tốt ở các tần số thấp, kém nhạy cảm với các tần số cao. Ta cần 1 cơ chế mapping tương tự như vậy.

![](image9.png)

Trước hết, ta bình phương các giá trị trong spectrogram thu được DFT power spectrum (phổ công suất). Sau đó, ta áp dụng 1 tập các bộ lọc thông dải Mel-scale filter trên từng khoảng tần số (mỗi filter áp dụng trên 1 dải tần xác định). Giá trị output của từng filter là năng lượng dải tần số mà filter đó bao phủ được. Ta thu được Mel-scale power spectrum. Ngoài ra, các filter dùng cho dải tần thấp thường hẹp hơn các filter dùng cho dải tần cao.

![](image10.png)

# 4. Cepstrum

## 4.1 Log

Mel filterbank trả về phổ công suất của âm thanh, hay còn gọi là phổ năng lượng. Thực tế rằng con . người kém nhạy cảm trong sự thay đổi năng lượng ở các tần số cao, nhạy cảm hơn ở tần số thấp. Vì vậy ta sẽ tính log trên Mel-scale power spectrum. Đeiefu này còn giúp giảm các biến thể âm thanh không đáng kể để nhận đạng giọng nói.

## 4.2 IDFT - Inverse DFT

Như đã mô tả ở phần trước, giọng nói của chúng ta có tần số F0 - tần số cơ bản và các formant F1,F2,F3,... Tần số F0 ở nam giới khoảng 125Hz, ở nữ là 210Hz, đặc trưng cho cao độ giọng nói ở từng người. Thông tin   về cao độ này không giúp ích trong nhận dạng giọng nói, nên ta cần tìm cách để loại thông tin về F0 đi, giúp các mô hình nhận dạng không bị phụ thuộc vào cao độ giọng từng người.

![](image11.png)

Trong hình này, tín hiệu chúng ta thu được là đồ thị 3, nhưng thông tin quan trọng chúng ta cần là phần 2, thông tin cần loại bỏ là phần1. Để loại bỏ đi thông tin về F0, ta làm 1 bước biến đổi Fourier ngược (IDFT) về miền thời gian, ta thu được Cepstrum. Nếu để ý kỹ, ta sẽ nhận ra rằng tên gọi "cepstrum" thực ra là đảo ngược 4 chữ cái đầu của spectrum.

Khi đó, với Cepstrum thu được, phần thông tin liên quan tới F0 và phần thông tin liên quan tới F1,F2,F3 ... nằm tách biệt nhau như 2 phần khoanh tròn trong hình 4. Ta chỉ đơn giản lấy thông tin trong đoạn đầu của cepstum (phần được khoanh tròn to trong hình 4). Để tính MFCC, ta chỉ cần lấy 12 giá trị đầu tiên.

Phép biến do IDFT cũng tương đương với 1 phép biến đổi DCT discrete cosine transformation. DCT là 1 phép biến đổi trực giao. Về mặt toán học, phép biến đổi này tạo ra các uncorrelated features, có thể hiểu là các feature độc lập hoặc có độ tương quan kém với nhau. Trong các thuật toán ML, uncorrelated features thường cho hiệu quả tốt hơn. Như vậy sau bước này, ta thu được 12 Cepstral features.

# 5. MFCC

Như vậy, mỗi frame ta đã extract ra được 12 Cepstral features đã làm 12 features đầu tiên của MFCC. feature thứ 13 là năng lượng của frame đó, tính theo công thức:

$$
Energy = \sum^{t_{2}}_{t=t_{1}}{x^2[t]}
$$

Trong nhận dạng tiếng nói, thông tin về bối cảnh và sự thay đổi rất quan trọng. VD tại những điểm mở đầu hoặc kết thúc ở nhiều phụ âm, sự thay đổi này rất rõ rệt, có thể nhận dạng các âm vị dựa vào sự thay đổi này. 13 hệ số tiếp theo chính là đạo hàm bậc 1 (theo thời gian) của 13 feature đầu tiên. Nó chứa thông tin về sự thay đổi từ frame thứ $t$ đến $t + 1$. Công thức:

$$
d(t) = \frac{c(t + 1) - c(t - 1)}{2}
$$

Tương tự như vậy, 13 giá trị cuối của MFCC là sự thay đổi $d(t)$ theo thời gian - đạo hàm của $d(t)$, đồng thời là đạo hàm bậc 2 của $c(t)$. Công thức:

$$
b(t) = \frac{d(t + 1) - d(t - 1)}{2}
$$

Vậy, từ 12 cepstral features và power feature thứ 13, ta đạo hàm 2 lần và thu được 39 features. Đây chính là MFCC feature.